<a href="https://colab.research.google.com/github/19mca/Cognifyz-Tasks-/blob/main/Restaurant_Recommendation-Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the dataset
restaurant_data = pd.read_csv('/content/drive/MyDrive/Cognify/Dataset .csv')

In [5]:
# Preprocessing
restaurant_data.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [ ]:
# Check for missing values
missing_values = restaurant_data.isnull().sum()
print("Missing values in each column:\n", missing_values)

Missing values in each column:
 Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64


In [7]:

# Drop rows with missing values
restaurant_data = restaurant_data.dropna().reset_index(drop=True)

# Encoding categorical variables
restaurant_data['Cuisines'] = restaurant_data['Cuisines'].astype('category').cat.codes
restaurant_data['City'] = restaurant_data['City'].astype('category').cat.codes
restaurant_data['Rating text'] = restaurant_data['Rating text'].astype('category').cat.codes

# Display encoded dataset
print("Encoded dataset:\n",restaurant_data.head())

Encoded dataset:
    Restaurant ID         Restaurant Name  Country Code  City  \
0        6317637        Le Petit Souffle           162    73   
1        6304287        Izakaya Kikufuji           162    73   
2        6300002  Heat - Edsa Shangri-La           162    75   
3        6318506                    Ooma           162    75   
4        6314302             Sambo Kojin           162    75   

                                             Address  \
0  Third Floor, Century City Mall, Kalayaan Avenu...   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   

                                     Locality  \
0   Century City Mall, Poblacion, Makati City   
1  Little Tokyo, Legaspi Village, Makati City   
2  Edsa Shangri-La, Ortigas, Mandaluyong City   
3      SM Megamall, Ortigas, Mandaluyong City   
4      SM Megam

In [ ]:
# Scaling numerical features
scaler = MinMaxScaler()
restaurant_data[['Price range', 'Aggregate rating']] = scaler.fit_transform(
    restaurant_data[['Price range', 'Aggregate rating']])

# Selecting features for content-based filtering
features = ['Cuisines', 'Price range', 'Aggregate rating']

# Calculate the cosine similarity matrix based on selected features
similarity_matrix = cosine_similarity(restaurant_data[features])

In [ ]:
# Recommendation Function based on restaurant name
def recommend_restaurants_by_name(restaurant_name, top_n=5):
    """Recommends top N similar restaurants to the restaurant with the given name."""
    # Find the index of the restaurant with the given name
    try:
        restaurant_index = restaurant_data[restaurant_data['Restaurant Name'].str.contains(restaurant_name, case=False, na=False)].index[0]
    except IndexError:
        print(f"Restaurant '{restaurant_name}' not found in the dataset.")
        return

    similar_restaurants = list(enumerate(similarity_matrix[restaurant_index]))
    similar_restaurants = sorted(similar_restaurants, key=lambda x: x[1], reverse=True)

    print(f"Top {top_n} similar restaurants to '{restaurant_name}':\n")
    for i in range(1, top_n + 1):
        print(restaurant_data.iloc[similar_restaurants[i][0]]['Restaurant Name'])

In [ ]:
# Example usage with user input for restaurant name
restaurant_name = input("Enter a restaurant name: ")
recommend_restaurants_by_name(restaurant_name)

Enter a restaurant name: Ooma
Top 5 similar restaurants to 'Ooma':

Sambo Kojin
Riviera
Cafe Blue Tomato
Cafe 55 - Park Inn
The Gem Bar & Restaurant


In [ ]:
# User Preference Based Recommendation
def recommend_based_on_user_preferences(user_preference, top_n=5):
    """Recommends top N restaurants based on user preferences."""
    user_similarity = cosine_similarity([user_preference], restaurant_data[features])
    recommended_restaurants = list(enumerate(user_similarity[0]))
    recommended_restaurants = sorted(recommended_restaurants, key=lambda x: x[1], reverse=True)

    print(f"Top {top_n} recommended restaurants based on user preferences:\n")
    for i in range(top_n):
        print(restaurant_data.iloc[recommended_restaurants[i][0]]['Restaurant Name'])

In [ ]:
# Example user preferences
user_preference = [0, 0.5, 0.7]  # Example normalized values for Cuisines, Price range, Aggregate rating
recommend_based_on_user_preferences(user_preference)

Top 5 recommended restaurants based on user preferences:

Kabul Delhi
Afghan Indian
Burj Al Arab
Kream
Nando's
